In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import yfinance
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def yahoo_finance_GSPC_download():
    data: pd.DataFrame = yfinance.download("^GSPC", start="1997-01-01", end="2024-08-31", interval="1d")
    data.to_csv("^GSPC.txt")


def GSPC_():
    def graph_(data: pd.DataFrame, index_worst_days: pd.DataFrame.index):
        length = data.shape[0]
        index = np.linspace(0, length, 9).astype(int)
        index[-1] -= 1
        plt.figure("HW1", figsize=(10, 8), facecolor="lightyellow")

        plt.subplot(311)
        target = data["Close"]
        plt.plot(range(length), target, linewidth=1, label="S&P 500")
        index_containers = []
        for i, value in enumerate(data.index):
            if value in index_worst_days:
                index_containers.append(i)
                # print(f"{i}, {value}")
        for value in index_containers:
            plt.axvline(value, color='red', linestyle='-', linewidth=0.5, zorder=4, label="Crisis")  # 添加垂直分界线
        plt.xticks(index, labels=data.index[index])
        plt.ylabel("Closing Price (USD)")
        plt.xlabel("Date")
        plt.yticks(np.linspace(target.min(), target.max(), 10))
        plt.title("Tendency of Closing Price of S&P 500")
        plt.grid(linestyle="--", alpha=0.2)
        plt.legend(["S&P 500", "Crisis"], loc="upper left")

        plt.subplot(312)
        target = data["Return_Percentage"]
        plt.title("S&P 500 Daily Return")
        plt.plot(range(length), target, linewidth=0.2, label="Return in Percentage(%)")
        plt.ylabel("Daily Return Percentage(%)")
        plt.xticks(index, labels=data.index[index])
        plt.xlabel("Date")
        plt.yticks(np.linspace(target.min(), target.max(), 10).round(1))
        plt.grid(linestyle="--", alpha=0.2)
        plt.legend(loc="best")

        plt.subplot(313)
        plt.title("Histogram of Daily Return in Percentage")
        target = data["Return_Percentage"]
        counts, bins, patches = plt.hist(target, bins=50, facecolor="blue", edgecolor="white", zorder=3, log=True)
        plt.xticks([bins.min(), 0, bins.max()])
        # plt.xticks(bins)
        counts = counts.astype(int)
        bins = 0.5 * (bins[1:] + bins[:-1])
        for i in range(len(counts)):
            plt.text(bins[i], counts[i], counts[i], va="bottom", ha="center", fontsize=5)
        plt.axvline(0, color='red', linestyle='--', linewidth=1, zorder=4)  # 添加垂直分界线
        plt.annotate("line of x=0", xy=(0, 1), xytext=(0.01, 5), c="yellow",
                     arrowprops=dict(arrowstyle="->", linewidth=1, linestyle="--", color="yellow"), zorder=20)
        plt.grid(axis="y",linestyle="--", alpha=0.2, zorder=0)
        plt.ylabel("Frequency of Return")
        plt.xlabel("Daily Return(%)")
        # plt.tight_layout(pad=0.5)
        plt.subplots_adjust(left=0.1, right=0.95, top=0.95, bottom=0.07, hspace=0.35)
        plt.show()


    def worst_days(data: pd.DataFrame):
        worst_days_index = data["Return_Percentage"].sort_values().index[:15]
        target = (data.loc[worst_days_index, :]).sort_index()
        target["Diff"] = target["Yesterday"] - target["Close"]
        target = target[["Yesterday", "Close", "Diff", "Return_Percentage"]]
        print(target)
        return worst_days_index

    data = pd.read_csv("^GSPC.txt", index_col=["Date"], usecols=["Date", "Close"])
    data["Yesterday"] = data["Close"].shift(1)
    data["Return_Percentage"] = ((data["Close"] - data["Yesterday"]) / data["Close"]).round(3) * 100
    index = worst_days(data)
    graph_(data, index)

if __name__ == "__main__":
    GSPC_()


__main__
